[View in Colaboratory](https://colab.research.google.com/github/soharabhossain/MachineLearning/blob/master/Recommender_System_Collaborative_Filtering.ipynb)

Acquiring data.....

In [0]:

# # Acquiring the Data

# To acquire and extract the data, simply run the following Bash scripts:  
# Dataset acquired from [GroupLens](http://grouplens.org/datasets/movielens/). Lets download the dataset. To download the data, we will use **`!wget`**. To download the data, we will use `!wget` to download it from IBM Object Storage.  

get_ipython().system('wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip')
print('unziping ...')
get_ipython().system('unzip -o -j moviedataset.zip ')


Import libraries.....

In [0]:
# First, let's get all of the imports out of the way:


# Dataframe manipulation library
import pandas as pd

# Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np

import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')


Read data from file.

In [8]:
# Now let's read each file into their Dataframes:

#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies.csv')

#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings.csv')


# Let's also take a peek at how each of them are organized:

#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Lets do a little preprocessing to extract out the year information from the title of the movies and add the year informartion as a separate column in the dataframe.

In [9]:
# So each movie has a unique ID, a title with its release year along with it (Which may contain unicode characters) and several different genres in the same field. Let's remove the year from the title column and place it into its own one by using the handy [extract](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.extract.html#pandas.Series.str.extract) function that Pandas has.

# Let's remove the year from the __title__ column by using pandas' replace function and store in a new __year__ column.

#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)

#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)

#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')

#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())


# Let's look at the result!

movies_df.head()




,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


Let us drop the genre column as we don't need it in this example.

In [10]:
# With that, let's also drop the genres column since we won't need it for this particular recommendation system.

#Dropping the genres column
movies_df = movies_df.drop('genres', 1)


# Here's the final movies dataframe:

print('\n Dimension of the movies dataframe : ')
print(movies_df.shape)

print('\n First few entries of the movies dataframe : ')
print(movies_df.head())



(34208, 3)
   movieId                        title  year
0        1                    Toy Story  1995
1        2                      Jumanji  1995
2        3             Grumpier Old Men  1995
3        4            Waiting to Exhale  1995
4        5  Father of the Bride Part II  1995


Now check the Ratings dataframe.....drop the timestamp column as we don't need it.

In [11]:

# Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save on memory.

print('\n Dimension of the ratings dataframe : ')
print(ratings_df.shape)


print('\n First few entries of the ratings dataframe : ')
print(ratings_df.head())


#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)


# Here's how the final ratings Dataframe looks like:

print(ratings_df.head())




 Dimension of the ratings dataframe : 
(22884377, 4)

 First few entries of the ratings dataframe : 
   userId  movieId  rating   timestamp
0       1      169     2.5  1204927694
1       1     2471     3.0  1204927438
2       1    48516     5.0  1204927435
3       2     2571     3.5  1436165433
4       2   109487     4.0  1436165496
   userId  movieId  rating
0       1      169     2.5
1       1     2471     3.0
2       1    48516     5.0
3       2     2571     3.5
4       2   109487     4.0


# Collaborative Filtering

Now, time to start our work on recommendation systems. 
 
The first technique we're going to take a look at is called __Collaborative Filtering__,
 which is also known as __User-User Filtering__.
 As hinted by its alternate name, this technique uses other users to recommend items to the input user.
 It attempts to find users that have similar preferences and opinions as the input and then recommends items
 that they have liked to the input. There are several methods of finding similar users
 (Even some making use of Machine Learning), and the one we will be using here is going to be based on the __Pearson Correlation Function__.
 
# <img src="https://ibm.box.com/shared/static/1ql8cbwhtkmbr6nge5e706ikzm5mua5w.png" width=800px>
 
 
 The process for creating a User Based recommendation system is as follows:
  - Select a user with the movies the user has watched
  - Based on his rating to movies, find the top X neighbours 
  - Get the watched movie record of the user for each neighbour.
  - Calculate a similarity score using some formula
  - Recommend the items with the highest score
 
 


Take an user input ratings for a number of movies.......we would recommend a few movies to this user subsequently.

In [12]:
# Let's begin by creating an input user to recommend movies to:
 
# Notice: To add more movies, simply increase the amount of elements in the userInput. 
# Feel free to add more in! Just be sure to write it in with capital letters 
# if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .


userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies



,rating,title
0,5.0,"Breakfast Club, The"
1,3.5,Toy Story
2,2.0,Jumanji
3,5.0,Pulp Fiction
4,4.5,Akira


We need to see the actual movie ratings by all the user corresponding to the movies rated by the input user. Therefore, we need to extract the ratings of all user from the ratings dataframe.

In [13]:
# #### Add movieId to input user
# With the input complete, let's extract the input movies's ID's from the movies dataframe and add them into it.

# We can achieve this by first filtering out the rows that contain the input movies' title
# and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.

# Filtering out the movies by title...get the list of the movies rated by the input user.
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#print(inputId.head())

# Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies) # Now we get the records for only the movies rated by the input user.
#print(inputMovies.head())

# Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('year', 1)

#Final input dataframe
# If a movie you added in above isn't here, then it might not be in the original 
# dataframe or it might spelled differently, please check capitalisation.
print(inputMovies.head())


      movieId                title  year
0           1            Toy Story  1995
1           2              Jumanji  1995
293       296         Pulp Fiction  1994
1246     1274                Akira  1988
1885     1968  Breakfast Club, The  1985
   movieId                title  year  rating
0        1            Toy Story  1995     3.5
1        2              Jumanji  1995     2.0
2      296         Pulp Fiction  1994     5.0
3     1274                Akira  1988     4.5
4     1968  Breakfast Club, The  1985     5.0
   movieId                title  rating
0        1            Toy Story     3.5
1        2              Jumanji     2.0
2      296         Pulp Fiction     5.0
3     1274                Akira     4.5
4     1968  Breakfast Club, The     5.0


We need to find the user groups who have similar tastes....so first we find the information of all the users who has seen and rated the similar set of movies.  These users have **most similar 'taste'** in terms of selection of movies with the input-user.

In [35]:
# The users who has seen the same movies
# With the movieId's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.

#Filtering out users that have watched movies that the input-user has also watched
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
print(userSubset.head())


# We now group up the rows by userId.

# Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
# groupby object containing one group per user : specified by the userId
# every group contains the information about the name of the movies seen by and rating given by this particular user
userSubsetGroup = userSubset.groupby(['userId'])


# Let's look at one of the users, e.g. the one with userID=1130
print('\n Information of a particular user group : specified by the userId : \n')
print(userSubsetGroup.get_group(1130))


# Let's also sort these groups so the users that share the most movies in common with the input-user have higher priority. 
# This provides a richer recommendation since we won't go through every single user.

# Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

# Now lets look at the first user
print('\n Information of top-five-similar-users : specified by the userId : \n')
print(userSubsetGroup[0:5])



     userId  movieId  rating
19        4      296     4.0
441      12     1968     3.0
479      13        2     2.0
531      13     1274     5.0
681      14      296     2.0

 Information of a particular user group : specified by the userId : 

        userId  movieId  rating
104167    1130        1     0.5
104168    1130        2     4.0
104214    1130      296     4.0
104363    1130     1274     4.5
104443    1130     1968     4.5

 Information of top-five-similar-users : specified by the userId : 

[(75,       userId  movieId  rating
7507      75        1     5.0
7508      75        2     3.5
7540      75      296     5.0
7633      75     1274     4.5
7673      75     1968     5.0), (106,       userId  movieId  rating
9083     106        1     2.5
9084     106        2     3.0
9115     106      296     3.5
9198     106     1274     3.0
9238     106     1968     3.5), (686,        userId  movieId  rating
61336     686        1     4.0
61337     686        2     3.0
61377     686     

## Similarity of users to input user
 Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.  
 we're going to find out how similar each user is to the input through the __Pearson Correlation Coefficient__. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below. 
 
**Why Pearson Correlation?**
 
Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

 
# ![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

 
The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation. 

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.

 We will select a subset of users to iterate through. This limit is imposed because we don't want to waste too much time going through every single user.

Similarity score are to be computer over the ratings these users given to the movies.

In [37]:
# Select to hundred entries.
userSubsetGroup = userSubsetGroup[0:100]


# Now, we calculate the Pearson Correlation between input user and subset groups
# and store it in a dictionary, where the key is the userId and the value is the value of the pearson-coefficient

# Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

# For every user group in our subset
for name, group in userSubsetGroup:
    
    # Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    
    # Get the N for the formula
    nRatings = len(group)
    
    # Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]

    # And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    
    # Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    
    # Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    # If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


print(pearsonCorrelationDict.items())
print('\n--------------------------------\n')
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))

print('\n--------------------------------\n')
print(pearsonDF.head())


# The top x similar users to input user
# Now let's get the top 50 users that are most similar to the input.
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
print('\n--------------------------------\n')
print('\nTop 10 users : \n')
print(topUsers.head(10))



dict_items([(75, 0.8272781516947562), (106, 0.5860090386731182), (686, 0.8320502943378437), (815, 0.5765566601970551), (1040, 0.9434563530497265), (1130, 0.2891574659831201), (1502, 0.8770580193070299), (1599, 0.4385290096535153), (1625, 0.716114874039432), (1950, 0.179028718509858), (2065, 0.4385290096535153), (2128, 0.5860090386731196), (2432, 0.1386750490563073), (2791, 0.8770580193070299), (2839, 0.8204126541423674), (2948, -0.11720180773462392), (3025, 0.45124262819713973), (3040, 0.89514359254929), (3186, 0.6784622064861935), (3271, 0.26989594817970664), (3429, 0.0), (3734, -0.15041420939904673), (4099, 0.05860090386731196), (4208, 0.29417420270727607), (4282, -0.4385290096535115), (4292, 0.6564386345361464), (4415, -0.11183835382312353), (4586, -0.9024852563942795), (4725, -0.08006407690254357), (4818, 0.4885967564883424), (5104, 0.7674257668936507), (5165, -0.4385290096535153), (5547, 0.17200522903844556), (6082, -0.04728779924109591), (6207, 0.9615384615384616), (6366, 0.65779

In [0]:
# Now, let's start recommending movies to the input user.


# Rating of selected users to all movies
# We're going to do this by taking the weighted average of the ratings of the movies using the
# Pearson Correlation as the weight.
# But to do this, we first need to get the movies watched by the users in our __pearsonDF__ from the ratings dataframe
# and then store their correlation in a new column called _similarityIndex".
# This is achieved below by merging of these two tables.


topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()


# Now all we need to do is simply multiply the movie rating by its weight (The similarity index),
# then sum up the new ratings and divide it by the sum of the weights.

# We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

# It shows the idea of all similar users to candidate movies for the input user:


#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()


#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()



#Creates an empty dataframe
recommendation_df = pd.DataFrame()

#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()


# Now let's sort it and see the top 20 movies that the algorithm recommended!

recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)


movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]



## Advantages and Disadvantages of Collaborative Filtering
 
** Advantages**
 * Takes other user's ratings into consideration
 * Doesn't need to study or extract information from the recommended item
 * Adapts to the user's interests which might change over time
 
** Disadvantages**
 * Approximation function can be slow
 * There might be a good number of users to approximate - huge data processing
 * Privacy issues when trying to learn the user's preferences
